# 残差网络（ResNet）
残差网络是卷积神经网络中十分重要、常用、好用且比较简单的一种网络。

### 残差块

（卷积层里的通道数是不是就对应着全连接层里的维度？）（好像是这样的）

In [1]:
import torch
from torch import nn
from torch.nn import functional as F
from d2l import torch as d2l

class Residual(nn.Module):
    # 参数分别为：输入的通道数、输出的通道数、是否需要1×1的卷积层、如果要1×1的卷积层的话步幅为多大
    def __init__(self, input_channels, num_channels, use_lxlconv=False, strides=1):
        super().__init__()
        # 第一个卷积层
        self.conv1 = nn.Conv2d(input_channels, num_channels, kernel_size=3, padding=1, stride=strides)
        # 第二个卷积层
        self.conv2 = nn.Conv2d(num_channels, num_channels, kernel_size=3, padding=1) # stride默认=1
        if use_lxlconv:
            # 1×1的卷积层，步长要与第一个卷积层的步长一致，才能保证最后可以相加
            self.conv3 = nn.Conv2d(input_channels, num_channels, kernel_size=1, stride=strides)
        else:
            self.conv3 = None
        # 两个批量规范化层
        self.bn1 = nn.BatchNorm2d(num_channels)  
        self.bn2 = nn.BatchNorm2d(num_channels)
        # 一个激活函数层
        self.relu = nn.ReLU(inplace=True)
        # inplace=True 指原地进行操作，操作完成后覆盖原来的变量，节省内存，但会造成进行梯度回归失败
        
    def forward(self, X):
        Y = F.relu(self.bn1(self.conv1(X)))
        # 进入第一个卷积层，随即进入一个批量规范化层，又随即进入激活函数
        
        Y = self.bn2(self.conv2(Y))
        # 进入第二个卷积层，随即批量规范化
        
        if self.conv3:
            X = self.conv3(X)
        Y += X             # 加上原数据
        return F.relu(Y)   # 加完后再次激活并返回
    
        
        

### 输入和输出形状一致

In [2]:
blk = Residual(3,3)  # 实例化一个输入和输出通道皆为3，不用1×1卷积，首个卷积步长为1（说明宽高都没有变化）的块
X = torch.rand(4,3,6,6)  
# 产生一个batch_size（批量大小）=4，channels（通道数）=3，h（高）=6，w（宽）=6 的数据输入。
Y = blk(X)
Y.shape


torch.Size([4, 3, 6, 6])

### 增加输出通道数的同时，减半输出的宽和高

In [3]:
blk2 = Residual(3, 6, use_lxlconv=True, strides=2)
# 输入通道数为3，输出通道数加倍为6，同时步长加倍为2，使宽高减半。
Y = blk2(X)
Y.shape



torch.Size([4, 6, 3, 3])

### 只增加输出通道数，不改变宽高

In [4]:
blk3 = Residual(3, 6, use_lxlconv=True, strides=1)
Y = blk3(X)
Y.shape

torch.Size([4, 6, 6, 6])

### ResNet模型

In [5]:
# 首先，预处理部分的块 b1，和 GoogleNet 的 b1 是一样的
# 一个7*7卷积层，一个批量规范化层，一个激活函数，一个3*3最大汇聚层
b1 = nn.Sequential(nn.Conv2d(1,64,kernel_size=7,stride=2,padding=3),
                   nn.BatchNorm2d(64),nn.ReLU(),
                   nn.MaxPool2d(kernel_size=3,stride=2,padding=1))

# 定义大ResNet块
# num_residuals 大ResNet块里有几个块
# first_block=False 是否是第一个大ResNet块
def resnet_block(input_channels, num_channels, num_residuals,  first_block=False):
    blk = []  # 创建一个列表用来存放块
    for i in range(num_residuals):
        if i==0 and not first_block: 
            # 如果是第一个大ResNet块，且是该大ResNet块中的是第一个块，那么要在这个小块里对宽高减半
            blk.append(Residual(input_channels, num_channels, use_lxlconv=True,strides=2))
        else:
            blk.append(Residual(num_channels, num_channels))
    return blk

b2 = nn.Sequential(*resnet_block(64,64,2,first_block=True)) # 第一个大残差块
b3 = nn.Sequential(*resnet_block(64,128,2))
b4 = nn.Sequential(*resnet_block(128,256,2))
b5 = nn.Sequential(*resnet_block(256,512,2))
# 在预处理 b1 部分先进行一个宽高减半，在第一个大残差块那里不进行减半，后面的每一个大ResNet块的第一块都进行减半

# 最后，与 GoogleNet 一样，加入一个全局平均汇聚层和全连接层
net = nn.Sequential(b1,b2,b3,b4,b5,nn.AdaptiveAvgPool2d((1,1)), nn.Flatten(), nn.Linear(512,10))
# net = nn.Sequential(b1,b2,b3,b4,b5,nn.Flatten(),nn.Linear(512,10))


全局平均汇聚层的作用是在不改变通道数的情况下，将宽高都变为1，这样才能接入最后的全连接层。
不太确定的话可以用上面cell最下面注释掉的那行代码来验证一下

### 观察ResNet中不同模块的输入形状是如何变化的

In [6]:
X = torch.rand(size=(1,1,256,256))
for layer in net:
    X = layer(X)
    print(layer.__class__.__name__, 'output shape:\t', X.shape)
    


Sequential output shape:	 torch.Size([1, 64, 64, 64])
Sequential output shape:	 torch.Size([1, 64, 64, 64])
Sequential output shape:	 torch.Size([1, 128, 32, 32])
Sequential output shape:	 torch.Size([1, 256, 16, 16])
Sequential output shape:	 torch.Size([1, 512, 8, 8])
AdaptiveAvgPool2d output shape:	 torch.Size([1, 512, 1, 1])
Flatten output shape:	 torch.Size([1, 512])
Linear output shape:	 torch.Size([1, 10])
